In [1]:
from argparse import Namespace
from collections import Counter
import json
import os
import re
import string

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm_notebook

In [2]:
from typing import *

In [3]:
class ReviewVectorizer(object):
    def __init__(self):
        pass

In [4]:
class ReviewDataset(Dataset):
    def __init__(self, dataset:pd.DataFrame, vectorizer:ReviewVectorizer):
        self.dataset = dataset
        self.vectorizer = vectorizer
        
        train = dataset[dataset.split=="train"]
        validation = dataset[dataset.split=="val"]
        test = dataset[dataset.split=="test"]
        
        self._lookup_dict = {
            "train": (train, len(train)),
            "validation": (validation, len(validation)),
            "test": (test, len(test))
        }
        
        self.set_split("train")
    
    @classmethod
    def load_dataset_and_make_vectorizer(cls, )
        